# 📘 代理架构 11：元控制器

欢迎来到我们系列的第十一个notebook。今天，我们探索**元控制器**架构，这是一种高级模式，其中一个"控制器"代理管理和协调多个专家代理。

元控制器充当指挥者，根据任务需求决定调用哪个专家代理，并综合它们的输出以产生最终结果。

### 定义
一个**Meta-Controller** (or 路由器) 是一个监督 代理在一个多代理 系统 负责为analyzing incoming 任务和dispatching them来appropriate specialized sub-agent或工作流程。它acts 作为一个intelligent routing layer, deciding which tool或expert is最好的suited为job在hand.

### 高级工作流程

1. **Receive Input:** 系统 receives 一个user request.
2. **Meta-Controller 分析:** Meta-Controller 代理 examines request's intent, 复杂的ity,和content.
3. **Dispatch来Specialist:** 基于它的分析, Meta-Controller selects最好的specialist 代理 (e.g., 'Researcher', 'Coder', 'Chatbot')从一个predefined pool.
4. **Execute Task:** chosen specialist 代理 executes task和generates 一个result.
5. **Return 结果：** result从specialist 被重新turned来user. 在more 复杂的 工作流程s,控制might return来Meta-Controller为further steps或monitoring.

### 何时使用/应用场景
* **Multi-Service AI Platforms:** 一个single entry point为一个platform 那个offers diverse services like document 分析,数据visualization,和creative writing.
* **Adaptive Personal Assistants:** 一个assistant 那个can switch between different modes或tools, such 作为managing你的calendar, searching web,或controlling smart home devices.
* **Enterprise 工作流程s:** Routing customer support tickets来right department (technical, billing, sales) 基于 ticket's content.

### 优点和缺点
* **优点：**
 * **Flexibility & Modularity:** Extremely easy来add新的capabilities通过simply adding 一个new specialist 代理和updating controller's routing logic.
 * **性能:** Allows为highly optimized, expert 代理instead的one jack-of-all-trades 模型 那个might 是mediocre在everything.
* **缺点：**
 * **Controller 作为一个Single Point的失败:** quality的entire 系统 hinges在controller's ability来route 任务 correctly. 一个poor routing decision leads一个suboptimal或incorrect outcome.
 * **Potential为Increased Latency:** extra step的routing 可以add 一个small amount的latency compared一个direct call一个single 代理.

## 阶段0：基础与设置

我们将安装 libraries和设置我们的environment. 我们将 need `langchain-tavily`用于我们的Research Agent's搜索tool.

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain-tavily

In [2]:
import os from typing import List, Dict, Any, Optional from dotenv import load_dotenv

# Pydanticfordata modeling from pydantic import BaseModel, Field

# LangChain components from langchain_nebius import ChatNebius from langchain_tavily import TavilySearch from langchain_core.prompts import ChatPromptTemplate

# LangGraph components from langgraph import StateGraph, END from typing_extensions import TypedDict

# 用于美观打印 from rich.console import Console from rich.markdown import Markdown

# --- API KeyandTracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Meta-Controller (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"] for var in required_vars:
 if var not in os.environ:
 print(f"警告: Environment variable {var} not set.")

print("环境变量已加载，追踪设置已完成。")

环境变量已加载，追踪设置已完成。


## 阶段1：构建专家代理

首先，我们将创建我们的专家代理团队。每个都将是一个具有特定角色的简单链，在研究员的情况下，还有一个工具。我们将把它们包装在节点函数中以在Lang图中使用。

In [3]:
console = Console()
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)
search_tool = TavilySearch(max_results=3)

# Define stateforoverall graph
class MetaAgentState(TypedDict):
 user_request: str
 next_agent_to_call: Optional[str]
 generation: str

# A helper factory function来创建specialist 代理 nodes
def create_specialist_node(persona: str, tools: list = None):
 """Factory来创建specialist 代理 node."""
 系统_prompt = f"你is一个specialist 代理withfollowing persona: {persona}. Respond directlyandconcisely来用户的请求 basedinyour role."
 prompt = ChatPromptTemplate.from_messages([
 ("system", 系统_prompt),
 ("human", "{user_request}")
 ])
 
 if tools:
 chain = prompt | llm.bind_tools(tools)
 else:
 chain = prompt | llm
 
 def specialist_node(state: MetaAgentState) -> Dict[str, Any]:
 result= chain.invoke({"user_request": state['user_request']})
 return{"generation": result.content}
 
 returnspecialist_node

# 1. Generalist Agent Node
generalist_node = create_specialist_node(
 "你is一个friendlyandhelpful generalist AI assistant. You处理casual conversationand简单的 questions."
)

# 2. Research Agent Node
research_agent_node = create_specialist_node(
 "你is一位专业的研究员. 你必须使用你的搜索工具 查找信息以回答用户的问题.",
 tools=[search_tool]
)

# 3. Coding Agent Node
coding_agent_node = create_specialist_node(
 "你is一位专业的Python程序员。 你的任务is根据用户的请求编写简洁、高效的Python代码. 仅提供代码，用markdown代码块包装，并提供最少的说明."
)

print("Specialist 代理defined successfully.")

Specialist 代理定义成功.


## 阶段2： 构建 Meta-Controller

这是 brain我们的系统. Meta-Controller 是一个LLM-powered node whose only job is来decide which specialist来route请求to. quality其prompt 是critical为系统's performance.

In [4]:
# Pydantic modelforcontroller's routing decision
class ControllerDecision(BaseModel):
 next_agent: str = Field(description="name的specialist 代理来call next. 必须be one的['Generalist', 'Researcher', 'Coder'].")
 reasoning: str = Field(description="A brief reasonforchoosing next 代理.")

def meta_controller_node(state: MetaAgentState) -> Dict[str, Any]:
 """central controller 那个decides which specialist来call."""
 console.print("--- 🧠 元控制器分析请求 ---")
 
 # Define specialistsandtheir descriptionsforcontroller
 specialists = {
 "Generalist": "Handles casual conversation, greetings,and简单的 questions.",
 "Researcher": "答案s问题about recent events, 复杂的 topics,oranything requiring up-to-date informationfromweb.",
 "Coder": "Writes Python code basedin一个user's specification."
 }
 
 specialist_descriptions = "\n".join([f"- {name}: {desc}"forname, descinspecialists.items()])
 
 prompt = ChatPromptTemplate.from_template(
 f"""你ismeta-controllerfor一个多代理 AI 系统. 你的工作isto分析用户的请求androute it来most appropriate specialist 代理.

Here are available specialists:
{specialist_descriptions}

Analyze following用户requestandchoose最好的specialist来handle it. Provide your decisioninrequired format.

User 请求: "{{user_request}}""""
 )
 
 controller_llm = llm.with_structured_output(ControllerDecision)
 chain = prompt | controller_llm
 
 decision = chain.invoke({"user_request": state['user_request']})
 console.print(f"[yellow]Routing decision:[/yellow] Send来[bold]{decision.next_agent}[/bold]. [italic]Reason: {decision.reasoning}[/italic]")
 
 return{"next_agent_to_call": decision.next_agent}

print("Meta-Controller node defined successfully.")

Meta-Controller node 定义成功.


## 阶段3： Assembling和Running 图

Now 我们将 使用Lang图来wire everything together. graph 将start与Meta-Controller,和then, based在its decision, 一个conditional edge 将route state来correct specialist node. 之后 specialist runs, graph 将end.

In [5]:
# Build graph
workflow = StateGraph(MetaAgentState)

# Add nodesforcontrollerandeach specialist
workflow.add_node("meta_controller", meta_controller_node)
workflow.add_node("Generalist", generalist_node)
workflow.add_node("Researcher", research_agent_node)
workflow.add_node("Coder", coding_agent_node)

# Set entry point
workflow.set_entry_point("meta_controller")

# Define conditional routing logic
def route_to_specialist(state: MetaAgentState) -> str:
 """Reads controller's decisionandreturn name的node来route to."""
 returnstate["next_agent_to_call"]

workflow.add_conditional_edges(
 "meta_controller",
 route_to_specialist,
 {
 "Generalist": "Generalist",
 "Researcher": "Researcher",
 "Coder": "Coder"
 }
)

# in...之后any specialist runs,过程ends
workflow.add_edge("Generalist", END)
workflow.add_edge("Researcher", END)
workflow.add_edge("Coder", END)

meta_agent = workflow.compile()
print("Meta-Controller 代理 Graph编译成功.")

Meta-Controller 代理 图编译成功.


## 阶段4： 演示nstration

让我们用各种提示测试我们的元控制器，看看它是否正确地将它们分派给正确的专家.

In [6]:
def run_agent(query: str):
 result= meta_agent.invoke({"user_request": query})
 console.print("\n[bold]最终response：[/bold]")
 console.print(Markdown(result['generation']))

# 测试1： 应该be routed来Generalist
console.print("--- 💬 测试1： General Conversation ---")
run_agent("Hello, how are you today?")

# 测试2： 应该be routed来Researcher
console.print("\n--- 🔬 测试2： Research Question ---")
run_agent("What were NVIDIA's latest financial results?")

# 测试3： 应该be routed来Coder
console.print("\n--- 💻 测试3： Coding Request ---")
run_agent("Can you write me 一个python function来calculate nth fibonacci number?")

--- 💬 测试1： General Conversation ---


--- 🧠 元控制器分析请求 ---
Routing decision: Send来Generalist. Reason: user's请求is 一个simple greeting, which falls under category的casual conversation handledpassGeneralist 代理.



最终响应：


Hello there! 如何可以I help你today?


--- 🔬 测试2： Research Question ---


...

NVIDIA's latest financial results, 为the quarter ending 在April 2024, 是exceptionally strong. They reported revenue 的$26.04 billion, 一个significant increase year-over-year, driven largely 通过their Data Center revenue which hit 一个record $22.6 billion. Their GAAP earnings per diluted share 是$5.98.


--- 💻 测试3： Coding Request ---


--- 🧠 元控制器分析请求 ---
Routing decision: Send来Coder. Reason:用户is explicitly askingfor一个Python function, which是一个coding task. Coder 代理是specialistforthis.



最终响应：


```python
def fibonacci(n):
 """Calculates nth Fibonacci number."""
 如果n <= 0:
 return 0
 elif n == 1:
 return 1
 else:
 a, b = 0, 1
为_ 在range(2, n + 1):
 a, b = b, 一个+ b
 return b
```

## 总结

在这个notebook中，我们have successfully implemented 一个**Meta-Controller** 架构. Our tests clearly 演示its primary function: acting 作为一个intelligent和dynamic router.

1. 简单的 greeting 是correctly identified和sent来**Generalist**.
2. 查询about recent financial news 是dispatched来**Researcher**, which used its搜索tool来fetch up-to-date information.
3. request为一个code snippet 是routed来**Coder**, which provided 一个well-formatted和correct function.

这种模式 是exceptionally 强大的为building scalable和maintainable AI 系统。 通过separating concerns, each specialist 可以be improved independently without affecting others. 系统's overall intelligence 可以be enhanced simply通过adding new, more capable specialists和making Meta-Controller aware的them. 虽然controller itself represents 潜在的 bottleneck,它的role 作为一个flexible orchestrator 是一个cornerstone的dvanced 代理ic design.